In [ ]:
import math
import pandas as pd
import seaborn as sns 
import tellurium as te
import matplotlib.pyplot as plt

fn_ant = '../model/PBK PFAS generic.ant'

generic_PFAS_model = te.loada(fn_ant)

# Helper function for plotting simulation results
def plotSimulationResults(res, selections):
    fig, axes = plt.subplots(3,math.ceil((len(selections) - 1)/3),figsize=(10,6))
    axes = axes.flatten()
    for ax in axes:
        ax.set(xlabel='time') 
    labels = selections
    for i in range(1, len(selections)):
        sid = selections[i]
        axes[i-1].plot(res['time'], res[sid])
        axes[i-1].set(ylabel=f"Amount", title=labels[i])
    fig.tight_layout()

def loadParametrisation(model, filename):
    df = pd.read_csv(filename)
    df['Value'] = df['Value'].astype(float)
    for index, row in df.iterrows():
        model[str(row['Parameter'])] = row['Value']
        
def run_generic_PFAS_model(rr_model, plot_params, start=0, end=29200, stepsize=14600):
    sim_result = rr_model.simulate(0, 29200, 14600, plot_params)
    sim_result = pd.DataFrame(sim_result, columns = sim_result.colnames)
    df_sim = pd.melt(sim_result, id_vars=['age'], value_vars=plot_params, var_name='quantity', value_name='value')
    g = sns.FacetGrid(df_sim, col='quantity', col_wrap=5, sharey=False, col_order=plot_params)
    g.map(sns.lineplot, 'age', 'value')
    g.add_legend()

plot_params = ['age', 'BW', 'Oraldose_PFAS', 'Dermaldose_PFAS', 'AG', 'AL','Asc','ASk','Aven','Cven','Cart','Afil','Adelay','Aurine']

# Simulate daily oral dosing via an event
# Specify the input
input_id = 'AG'

# Make sure A_gut is not constant and does not have boundary conditions
generic_PFAS_model.setInitAmount(input_id, 0)
generic_PFAS_model.setConstant(input_id, False)
generic_PFAS_model.setBoundary(input_id, False)

# Remove all events
for event_id in generic_PFAS_model.model.getEventIds():
    generic_PFAS_model.removeEvent(event_id)

# bw = 70
# daily_intake = 3.55 * 0.011 * bw # ug/d (0.00355 mg/l * 11 mL/kg/day)
Oraldose = 0.0001 # ug/kg bw

days_of_exposure = 30 * 365
days_after_exposure = 10 * 365
num_days = days_of_exposure + days_after_exposure

# Set chemical parameters
loadParametrisation(rr_model, '../Parametrisations/PFOA_Human.csv')
# generic_PFAS_model.QLC = 0.25

# Set physiological parameters
# generic_PFAS_model.BW = bw
# generic_PFAS_model.AGE = 30

# Create a repeating daily oral dosing
eid = f"oral_daily_exposure"
generic_PFAS_model.addEvent(eid, False, f"time % 1 == 0 && time < {days_of_exposure}", False)
generic_PFAS_model.addEventAssignment(eid, input_id, f"{input_id} + {Oraldose}", False) # Dose needs to be multiplied with BW
generic_PFAS_model.regenerateModel(True, True)

# Simulate the PBPK model
results = generic_PFAS_model.simulate(0, num_days, num_days + 1)

# Plot the results
plotSimulationResults(results, generic_PFAS_model.timeCourseSelections)

# generic_PFAS_model.Dermconc = 10
# run_generic_PFAS_model(generic_PFAS_model, plot_params, start=0, end=29200, stepsize=14600)